In [15]:
import pandas as pd
import numpy as np
import math

In [16]:
df_pitch_subs = pd.read_csv('pitching_subs.csv')
df_pitch_subs.head(50)

,GAME_ID,PIT_ID,INN_CT,HOME_TEAM,OUTS_CT,AWAY_SCORE_CT,HOME_SCORE_CT,onFirst,onSecond,onThird,SAVE_PIT_ID
0,ANA201004050,jepsk001,7,1,0,3,4,0,0,0,fuenb001
1,ANA201004050,rodnf001,8,1,0,3,4,0,0,0,fuenb001
2,ANA201004050,fuenb001,9,1,0,3,6,0,0,0,fuenb001
3,ANA201004050,craij001,5,0,2,3,4,1,0,1,fuenb001
4,ANA201004050,mijaj001,7,0,1,3,4,0,0,0,fuenb001
5,ANA201004050,neshp001,8,0,0,3,6,0,0,0,fuenb001
6,ANA201004060,palmm001,6,1,0,5,3,0,0,0,raucj001
7,ANA201004060,stokb001,8,1,0,5,3,0,0,0,raucj001
8,ANA201004060,duenb001,7,0,2,5,3,0,1,0,raucj001
9,ANA201004060,guerm001,8,0,0,5,3,0,0,0,raucj001


In [17]:
def save_sit(row):
    if row['HOME_TEAM'] == 1 and row['HOME_SCORE_CT'] > row['AWAY_SCORE_CT']:
        return True
    elif row['HOME_TEAM'] == 0 and row['HOME_SCORE_CT'] < row['AWAY_SCORE_CT']:
        return True
    else:
        return False

df_pitch_subs['SAVE_SIT'] = df_pitch_subs.apply(save_sit, axis=1)

In [18]:
df_pitch_subs[['GAME_ID', 'PIT_ID', 'INN_CT', 'HOME_TEAM', 'AWAY_SCORE_CT', 'HOME_SCORE_CT', 'SAVE_PIT_ID', 'SAVE_SIT']].head(5)

,GAME_ID,PIT_ID,INN_CT,HOME_TEAM,AWAY_SCORE_CT,HOME_SCORE_CT,SAVE_PIT_ID,SAVE_SIT
0,ANA201004050,jepsk001,7,1,3,4,fuenb001,True
1,ANA201004050,rodnf001,8,1,3,4,fuenb001,True
2,ANA201004050,fuenb001,9,1,3,6,fuenb001,True
3,ANA201004050,craij001,5,0,3,4,fuenb001,False
4,ANA201004050,mijaj001,7,0,3,4,fuenb001,False


In [19]:
df_pitch_subs = df_pitch_subs[df_pitch_subs['SAVE_SIT'] == True].reset_index()

In [20]:
df_pitch_subs[['GAME_ID', 'PIT_ID', 'INN_CT', 'HOME_TEAM', 'AWAY_SCORE_CT', 'HOME_SCORE_CT', 'SAVE_PIT_ID', 'SAVE_SIT']].head(5)

,GAME_ID,PIT_ID,INN_CT,HOME_TEAM,AWAY_SCORE_CT,HOME_SCORE_CT,SAVE_PIT_ID,SAVE_SIT
0,ANA201004050,jepsk001,7,1,3,4,fuenb001,True
1,ANA201004050,rodnf001,8,1,3,4,fuenb001,True
2,ANA201004050,fuenb001,9,1,3,6,fuenb001,True
3,ANA201004060,duenb001,7,0,5,3,raucj001,True
4,ANA201004060,guerm001,8,0,5,3,raucj001,True


In [21]:
df_next_pitcher = df_pitch_subs.shift(-1).reset_index()
df_next_pitcher[['GAME_ID', 'PIT_ID', 'INN_CT', 'HOME_TEAM', 'AWAY_SCORE_CT', 'HOME_SCORE_CT', 'SAVE_PIT_ID', 'SAVE_SIT']].head(100)

,GAME_ID,PIT_ID,INN_CT,HOME_TEAM,AWAY_SCORE_CT,HOME_SCORE_CT,SAVE_PIT_ID,SAVE_SIT
0,ANA201004050,rodnf001,8.0,1.0,3.0,4.0,fuenb001,True
1,ANA201004050,fuenb001,9.0,1.0,3.0,6.0,fuenb001,True
2,ANA201004060,duenb001,7.0,0.0,5.0,3.0,raucj001,True
3,ANA201004060,guerm001,8.0,0.0,5.0,3.0,raucj001,True
4,ANA201004060,raucj001,9.0,0.0,5.0,3.0,raucj001,True
5,ANA201004070,guerm001,8.0,0.0,4.0,1.0,raucj001,True
6,ANA201004070,raucj001,9.0,0.0,4.0,1.0,raucj001,True
7,ANA201004080,neshp001,6.0,0.0,3.0,1.0,None,True
8,ANA201004080,mijaj001,7.0,0.0,3.0,1.0,None,True
9,ANA201004080,craij001,8.0,0.0,6.0,1.0,None,True


In [22]:
df_pitch_subs['made_save'] = None
df_pitch_subs.head(5)

,index,GAME_ID,PIT_ID,INN_CT,HOME_TEAM,OUTS_CT,AWAY_SCORE_CT,HOME_SCORE_CT,onFirst,onSecond,onThird,SAVE_PIT_ID,SAVE_SIT,made_save
0,0,ANA201004050,jepsk001,7,1,0,3,4,0,0,0,fuenb001,True,None
1,1,ANA201004050,rodnf001,8,1,0,3,4,0,0,0,fuenb001,True,None
2,2,ANA201004050,fuenb001,9,1,0,3,6,0,0,0,fuenb001,True,None
3,8,ANA201004060,duenb001,7,0,2,5,3,0,1,0,raucj001,True,None
4,9,ANA201004060,guerm001,8,0,0,5,3,0,0,0,raucj001,True,None


In [25]:
for t1, t2 in zip(df_pitch_subs.iterrows(), df_next_pitcher.iterrows()):
    cur_index = t1[0]
    cur_row = t1[1]
    next_row = t2[1]
    made_save = None
    
    b = next_row['HOME_SCORE_CT'] <= next_row['AWAY_SCORE_CT']
    if cur_row['GAME_ID'] == next_row['GAME_ID'] and cur_row['HOME_TEAM'] == next_row['HOME_TEAM']:
        # in same game, and same team.
        if cur_row['HOME_TEAM'] == 1:
            # home team
            blew_lead = next_row['HOME_SCORE_CT'] <= next_row['AWAY_SCORE_CT']
            if blew_lead:
                inn_diff = next_row['INN_CT'] - cur_row['INN_CT']
                init_score_diff = cur_row['HOME_SCORE_CT'] - cur_row['AWAY_SCORE_CT']
                
                if (init_score_diff <= 3 and inn_diff >= 1) or (inn_diff >= 3) or (cur_row['HOME_SCORE_CT'] <= cur_row['AWAY_SCORE_CT'] + 2 + sum([cur_row['onFirst'], cur_row['onSecond'], cur_row['onThird']])):
                    made_save = False
                
            
        elif cur_row['HOME_TEAM'] == 0:
            # away team
            blew_lead = next_row['HOME_SCORE_CT'] >= next_row['AWAY_SCORE_CT']
            if blew_lead:
                inn_diff = next_row['INN_CT'] - cur_row['INN_CT']
                init_score_diff = cur_row['AWAY_SCORE_CT'] - cur_row['HOME_SCORE_CT']
                
                if (init_score_diff <= 3 and inn_diff >= 1) or (inn_diff >= 3) or (cur_row['AWAY_SCORE_CT'] <= cur_row['HOME_SCORE_CT'] + 2 + sum([cur_row['onFirst'], cur_row['onSecond'], cur_row['onThird']])):
                    made_save = False
    
    
    elif cur_row['PIT_ID'] == cur_row['SAVE_PIT_ID']:
            made_save = True
               

    df_pitch_subs.iloc[cur_index, df_pitch_subs.columns.get_loc('made_save')] = made_save
    
    
df_pitch_subs.head(50)

b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: True
b: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: False
b: False
b_1: False
b: True
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: False
b: True
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b: True
b

b: False
b_1: False
b: True
b: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b: True
b: True
b: True
b_2 False
b: False
b: True
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: False
b: True
b: False
b: False
b: True
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b: True
b_2 False
b: False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b: True
b: True
b_2 False
b: True
b: True
b_2 False
b: False
b: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b: False
b: True
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b: False
b: True
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: True
b: False
b: False
b_1: False
b: True
b: True
b_2 False
b: False

b_2 False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: True
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: False
b: True
b: False
b: True
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: False
b: True
b: True
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: False
b_1: False
b: True
b: False
b: True
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: True
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: True
b: False
b: False
b: False
b: True
b: Tr

b: False
b_1: False
b: False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: False
b: False
b_1: False
b: True
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b: True
b: True
b_2 False
b: True
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: True
b: False
b: False
b_1: False
b: False
b: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: True
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: True
b: False
b

b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: True
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_

...

b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: True
b: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: False
b:

b: False
b_1: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: True
b: False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: True
b: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: True
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b: False
b_1: Fal

b: False
b_1: False
b: False
b_1: False
b: False
b: False
b: False
b_1: False
b: True
b: True
b_2 False
b: False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: False
b: True
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: True
b: True
b_2 False
b: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b_1: False
b: True


b_2 False
b: False
b: False
b_1: False
b: True
b: False
b: False
b_1: False
b: True
b: False
b: True
b: True
b: True
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b: False
b_1: False
b: False
b: True
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b: False
b: True
b: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b_1: False
b: False
b_1: False
b: True
b: True
b: False
b: False
b: False
b_1: False
b: True
b: False
b: False
b_1: False
b: False
b_1: False
b: False
b: False
b_1: False
b: True
b: False
b: False
b: True
b: True
b_2 False
b: True
b_2 False
b: False
b: False
b: False
b_1: False
b: False
b: True
b: True
b_2 False
b: True
b_2 False
b: True
b_2 False
b: True
b: True
b_2 False
b: True
b_2 False

,index,GAME_ID,PIT_ID,INN_CT,HOME_TEAM,OUTS_CT,AWAY_SCORE_CT,HOME_SCORE_CT,onFirst,onSecond,onThird,SAVE_PIT_ID,SAVE_SIT,made_save
0,0,ANA201004050,jepsk001,7,1,0,3,4,0,0,0,fuenb001,True,None
1,1,ANA201004050,rodnf001,8,1,0,3,4,0,0,0,fuenb001,True,None
2,2,ANA201004050,fuenb001,9,1,0,3,6,0,0,0,fuenb001,True,True
3,8,ANA201004060,duenb001,7,0,2,5,3,0,1,0,raucj001,True,None
4,9,ANA201004060,guerm001,8,0,0,5,3,0,0,0,raucj001,True,None
5,10,ANA201004060,raucj001,9,0,0,5,3,0,0,0,raucj001,True,True
6,13,ANA201004070,guerm001,8,0,0,4,1,0,0,0,raucj001,True,None
7,14,ANA201004070,raucj001,9,0,0,4,1,0,0,0,raucj001,True,True
8,18,ANA201004080,neshp001,6,0,1,3,1,1,1,0,None,True,None
9,19,ANA201004080,mijaj001,7,0,0,3,1,0,0,0,None,True,None


In [26]:
df_blown_save = df_pitch_subs[df_pitch_subs['made_save'] == False]
df_blown_save.head()

,index,GAME_ID,PIT_ID,INN_CT,HOME_TEAM,OUTS_CT,AWAY_SCORE_CT,HOME_SCORE_CT,onFirst,onSecond,onThird,SAVE_PIT_ID,SAVE_SIT,made_save
